In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import login  # a python file that contains my login information
import time
import json

path = r'/Users/yibowang/Documents/Chromedriver/chromedriver'
login_page = 'https://www.lib.uwo.ca/cgi-bin/ezpauthn.cgi?url=http://global.factiva.com/en/sess/login.asp?xsid=S003WvtZWzk5DEs5DEmM9EpODaoOT7yMHmnRsIuMcNG1pRRQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA'
username = login.username
password = login.password

In [2]:
# Initiate a web browser and go to the url in the browser
options = webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome(path, options=options)
driver.get(login_page)

In [3]:
# log in to the website to create one session
user = driver.find_element_by_name('user')
user.send_keys(username)
pin = driver.find_element_by_name('pass')
pin.send_keys(password)
login_button = driver.find_element_by_xpath(
    '/html/body/center[2]/div/form/p[4]/input')
login_button.click()


In [4]:
# langding page: lp = "https://global-factiva-com.proxy1.lib.uwo.ca/sb/default.aspx?lnep=hp"
# time.sleep(50) this log in process roughly takes about 45 seconds
home = driver.find_element_by_link_text("Home")
home.click()

In [5]:
# time.sleep(10) this clicking process roughly takes about 10 seconds
viewall = driver.find_element_by_link_text("View All")
viewall.click()


In [ ]:
# time.sleep(15) this clicking process roughly takes about 18 seconds
html_doc = []
all_title = []
for index in range(1,60):
    xpath = '//*[@id="headlines"]/table/tbody/tr[' + str(index) + ']/td[3]/a' # print(xpath)
    headline = driver.find_element_by_xpath(xpath)
    headline.click()
    all_title.append(headline.text)
    # time.sleep(3) # previously was set to 2 seconds but it might be too short since loading takes more than that
    
    # use bs4 to parse text information 
    article_html = driver.page_source
    article = BeautifulSoup(article_html, "html.parser")
    html_doc.append(article)

In [8]:
wsj = []
for c in html_doc:
    if c.find("div", {"class": "author"}) != None:   
        news = {}
        news["paragraph"] = c.find_all("p", {"class": "articleParagraph enarticleParagraph"}) # a list of paragraphs
        news["title"] = c.find("span", {"class": "enHeadline"})
        # print(news["title"]) for illustrating purpose
        author = c.find("div", {"class": "author"})
        news["author"] = author
        news["others"] = author.find_next_siblings("div") # information about the article; others[2] is date
        wsj.append(news)

In [9]:
driver.close()

In [10]:
journal = []
for c in html_doc:
    if c.find("span", {"class": "enHeadline"}) != None:   
        news = {}
        title = c.find("span", {"class": "enHeadline"})
        news["title"] = title.text
        
        hd = c.find("div", {"id": "hd"})
        others = hd.find_next_siblings("div") # information about the article; others[2] is date
        news["author"] = others[0].text
        news["wordcount"] = others[2].text
        news["date"] = others[3].text
        
        paragraphs = c.find_all("p", {"class": "articleParagraph enarticleParagraph"}) # a list of paragraphs
        news["paragraph"] = [p.text for p in paragraphs]
        

        journal.append(news)

In [ ]:
"""for i in wsj:
    print(i["title"].text)
    print(i["others"][0].text)
    print(i["others"][2].text)
    print("-----------------------------------")
    for paragraph in i["paragraph"]:
        print(paragraph.text)
        print("\n")
    print("***************************************")"""

In [ ]:
for read in journal[31:37]:
    print(read['title'])
    print(read['author'])
    print(read['date'])
    print("-----------------------------------")
    for i in read['paragraph']:
        print(i)
        print('\n')
    print("***************************************")

In [11]:
## this is not good;
"""with open("pywsj_content.json", "w") as outfile:
    for news_article in journal:
        json.dump(news_article, outfile)
        outfile.write('\n')"""

## simple load the list of dictionaries into json
with open("pywsj_may14.json", "w") as output:
    json.dump(journal, output)